In [1]:
!nvidia-smi

Wed Nov 29 12:02:13 2023       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.12             Driver Version: 535.104.12   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   37C    P8               9W /  70W |      2MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [21]:
torch.cuda.is_available()

True

In [22]:
torch.cuda.device_count()

4

In [ ]:
!pip install --no-cache torch==2.1.0 --index-url https://download.pytorch.org/whl/cu121

In [ ]:
pip install transformers accelerate

In [16]:
!pip install gradio -q
!pip install --upgrade --force-reinstall  typing-extensions

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchvision 0.14.1+cu116 requires torch==1.13.1, but you have torch 2.1.0+cu121 which is incompatible.


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


  Using cached typing_extensions-4.8.0-py3-none-any.whl (31 kB)
  Attempting uninstall: typing-extensions
    Found existing installation: typing_extensions 4.8.0
    Uninstalling typing_extensions-4.8.0:
      Successfully uninstalled typing_extensions-4.8.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchvision 0.14.1+cu116 requires torch==1.13.1, but you have torch 2.1.0+cu121 which is incompatible.


In [2]:
from transformers import AutoTokenizer

import torch
import transformers

In [3]:
!huggingface-cli login --token HUGGINGFACE_TOKEN

Token will not been saved to git credential helper. Pass `add_to_git_credential=True` if you want to set the git credential as well.
Token is valid (permission: read).
Your token has been saved to /home/jovyan/.cache/huggingface/token
Login successful


In [4]:
!huggingface-cli whoami

girishspatil


In [5]:
model_id = 'meta-llama/Llama-2-7b-chat-hf'

# begin initializing HF items, need auth token for these
hf_auth = 'HUGGINGFACE_TOKEN'

In [6]:
tokenizer = transformers.AutoTokenizer.from_pretrained(
    model_id,
    token=hf_auth
)

In [7]:
generate_text = transformers.pipeline(
    task='text-generation', # we pass model parameters here too
    model=model_id, 
    torch_dtype=torch.float16,
    device_map='auto'
)

2023-11-28 10:14:11.373548: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-11-28 10:14:11.557559: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-11-28 10:14:12.250631: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/nvidia/lib:/usr/local/nvidia/lib64
2023-11-28 10:14:12.250798: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinf

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [19]:
def predict(prompt: str) -> None:
    """
    get response from llama2 model pipeline

    Parameters:
        promt: The user's input/question to the model

    Returns:
        None: No return output, but prints the model response as part of the function
    
    """

    response = generate_text(
        prompt,
        do_sample=True,
        top_k=10,
        num_return_sequences=1,
        eos_token_id=tokenizer.eos_token_id,
        max_length=512
    )

    print("response :",response[0]["generated_text"])

In [20]:
predict("Explain me the difference between Data Lakehouse and Data Warehouse.")

response : Explain me the difference between Data Lakehouse and Data Warehouse.

A data lakehouse is a centralized repository that stores raw, unprocessed data in its native format, while a data warehouse is a structured repository that stores data in a schema-defined format.

Here are some key differences between the two:

1. Data Structure: A data lakehouse stores data in its raw, unprocessed form, while a data warehouse stores data in a structured format, with a predefined schema.
2. Data Processing: A data lakehouse does not perform any data processing, while a data warehouse performs various data processing tasks, such as data cleansing, transformation, and aggregation.
3. Data Access: A data lakehouse provides direct access to raw data, while a data warehouse provides structured data through a query interface.
4. Scalability: A data lakehouse is designed to handle large volumes of raw data and can scale horizontally by adding more nodes, while a data warehouse can become bottlene